<h1 style="font-size:24px; color:red; text-decoration:underline; font-weight:bold;">
  Import
</h1>


In [3]:
import logging
import os
import sys
import tempfile
from glob import glob

import torch
from PIL import Image
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch, DataLoader
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
)
from monai.visualize import plot_2d_or_3d_image

from transform import TrainTransforms,ValidTransforms
from loaders import LotusDataModule
import argparse
import glob 
import matplotlib.pyplot as plt

from model import UNetLightningModule
import argparse


from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import NeptuneLogger
from pytorch_lightning.strategies.ddp import DDPStrategy

<h1 style="font-size:24px; color:red; text-decoration:underline; font-weight:bold;">
  Test Loader
</h1>

In [4]:
import matplotlib.pyplot as plt

train_csv = "/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/training/label2/train.csv"
val_csv = "/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/training/label2/valid.csv"
test_csv = "/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/training/label2/test.csv"
batch_size = 2
num_workers = 4
img_colums = "mri"
seg_column = "seg"
# define transforms for image and segmentation
train_transforms = TrainTransforms()
val_transforms = ValidTransforms()
# val_transforms =None

loader = LotusDataModule(train_csv,val_csv,test_csv,batch_size,num_workers,img_colums,seg_column,train_transforms,val_transforms)
loader.setup()

train_ds = loader.train_dataset()
val_ds = loader.val_dataset()

val_loader = loader.val_dataloader()
train_loader = loader.train_dataloader()

# print("train_ds : ",train_ds)
# # print(train_loader)
# check_data = monai.utils.misc.first(val_loader)
# print(check_data["img"].shape, check_data["seg"].shape)

# img = check_data["img"][0][0]  # Assuming single channel and first sample in batch
# seg = check_data["seg"][0][0]
# plt.figure("Check Image", (12, 6))
# plt.subplot(1, 2, 1)
# plt.title("Image")
# plt.imshow(img.numpy(), cmap="gray")
# plt.subplot(1, 2, 2)
# plt.title("Segmentation")
# plt.imshow(seg.numpy(), cmap="gray")
# plt.show()

model = UNetLightningModule()

trainer = Trainer(
    logger=None,
    log_every_n_steps=100,
    max_epochs=300,
    max_steps=-1,
    callbacks=None,
    accelerator='gpu', 
    devices=torch.cuda.device_count(),
    strategy="ddp_notebook",
    reload_dataloaders_every_n_epochs=1
)


# trainer.fit(model, datamodule=concat_dat
trainer.fit(model, datamodule=loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type     | Params
-------------------------------------------
0 | model         | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]data_dict : data_dict :  data_dict : {'img': '/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/a1_MRI_2D/valid/B020_MR_Crop_slice_040.png', 'seg': '/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/a2_mri_Seg2D_label2/valid/B020_MR_Seg_clean_slice_040.png'}  {'img': '/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/a1_MRI_2D/valid/B015_MR_Crop_slice_071.png', 'seg': '/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/a2_mri_Seg2D_label2/valid/B015_MR_Seg_clean_slice_071.png'}{'img': '/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/a1_MRI_2D/valid/B022_MR_Crop_slice_013.png', 'seg': '/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/a2_mri_Seg2D_label2/valid/B022_MR_Seg_clean_slice_013.png'}


data_dict : data_dict : data_dict :    data_dict : {'img': '/home/lucia/Documents/Gaelle/Data/MultimodelReg/Segmentation/a1_MRI_2D/valid/B020_MR_Crop_slice_057.png', 'seg': '/home

ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/home/lucia/APP/miniconda3/envs/training_mri_cbct2/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", line 75, in _wrap
    fn(i, *args)
  File "/home/lucia/APP/miniconda3/envs/training_mri_cbct2/lib/python3.9/site-packages/pytorch_lightning/strategies/launchers/multiprocessing.py", line 173, in _wrapping_function
    results = function(*args, **kwargs)
  File "/home/lucia/APP/miniconda3/envs/training_mri_cbct2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 580, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/home/lucia/APP/miniconda3/envs/training_mri_cbct2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 987, in _run
    results = self._run_stage()
  File "/home/lucia/APP/miniconda3/envs/training_mri_cbct2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 1031, in _run_sta